In [ ]:
import numpy as np
from skimage import io
from utils import grainPreprocess, grainMark, SEMDataset, grainDraw
from numpy.lib.stride_tricks import sliding_window_view

from matplotlib import pyplot as plt

from skimage import io, color, filters, morphology, util
from skimage.measure import EllipseModel
from skimage.color import rgb2gray
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball
from skimage.measure import approximate_polygon
from skimage import transform
import copy
from PIL import Image, ImageDraw, ImageFilter, ImageOps

from matplotlib import cm
import networkx as nx
from tqdm.notebook import tqdm

from bresenham import bresenham
import pandas as pd

from shapely.geometry import Polygon, LineString
import rasterio.features
import matplotlib.pyplot as plt

from networkx.drawing import draw_networkx
import itertools

from multiprocessing import Lock, Process, Queue, current_process
import multiprocessing
import pickle

import cv2
from mpire import WorkerPool
import time
import shutil
from pathlib import Path
import json

import psutil
import os
import subprocess

# Find and plot all nodes

In [ ]:
def align_figures(orig_img_padded, tol):
    cnts = grainMark.get_contours(orig_img_padded,tol=tol)
    cnts = [np.array(cnt)[:-1] for cnt in cnts if len(cnt)>2]
    cnts_adj = [cnt.reshape((-1,1,2)) for cnt in cnts ]
    
    white_img = np.full((orig_img_padded.shape[0],orig_img_padded.shape[1],3),255)
    white_img = np.ascontiguousarray(white_img, dtype=np.uint8)
    img_viz = cv2.drawContours(white_img,cnts_adj,-1,(127,127,127),-1).astype(np.uint8)
    
    return img_viz, cnts

def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 255)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value


def preprocess_image(binary):

    grad = abs(filters.rank.gradient(binary, morphology.disk(1)))
    bin_grad = (1 - binary + grad) * 127
    bin_grad = np.clip(bin_grad, 0, 255).astype(np.uint8)

    return bin_grad
        

def preprocess_image(image, r=2, border = 30, border_node_eps=10, tol = 5, plot=False):

    border_eps = border + border_node_eps

    if len(image.shape)==3:
        image = color.rgb2gray(image)
    
    image = filters.rank.median(image, morphology.disk(3))

    global_thresh = filters.threshold_otsu(image)
    image = image > global_thresh
    binary = image*255
    binary = binary.astype(np.uint8)
    
    orig_img_padded = np.pad(binary, border, pad_with)
    tmp_img = SEMDataset.preprocess_image(orig_img_padded)
    
    img_aligned,cnts = align_figures(tmp_img,tol)
    
    img_shape=np.array(img_aligned.shape)
    
    # coord2index
    image_nodes_coord2nodes_index={}
    nodes_index2global_nodes_coord={}
    num_of_nodes=0
    
    for points in reversed(cnts):
        for point in points:
            x,y = point[0],point[1]
            image_nodes_coord2nodes_index[(y,x)]=num_of_nodes
            nodes_index2global_nodes_coord[num_of_nodes]=(y,x)
            num_of_nodes+=1
    
    # entry points
    entry_nodes=[]
    
    entry_dict={}
    y_entry_max=0
    
    for points in cnts:
        for point in points:
            if point[1]<border_eps:
                x,y = point[0],point[1]
                # condition to make end exit poits below start points 
                if y_entry_max<y:
                    y_entry_max=y
                
                index=image_nodes_coord2nodes_index[(y,x)]
                entry_dict[index]=1
                entry_nodes.append(index)
    
    # exit points
    exit_nodes=[]
    exit_dict={}
    
    for points in cnts:
        for point in points:
            # if (point[0] < border_eps or img_shape[0] - point[0] < border_eps or img_shape[1] - point[1] < epborder_epss) and point[1]>y_entry_max:
            if (img_shape[0] - point[1] < border_eps):
                x,y = point[0],point[1]
                index=image_nodes_coord2nodes_index[(y,x)]
                exit_nodes.append(index)
                exit_dict[index]=1
    
    img_drawings = copy.copy(Image.fromarray(img_aligned)).convert('RGB')
    img_drawings = grainDraw.draw_contours(img_drawings, cnts=cnts, color_corner=(0, 139, 139), color_line = (255, 140, 0),corners = True)
    draw = ImageDraw.Draw(img_drawings)
    
    # entry blue
    for key in entry_nodes:
        x,y=nodes_index2global_nodes_coord[key]
        draw.ellipse((y - r, x - r, y + r, x + r), fill=(0,0,255), width=1)
        
    # exit red
    for key in exit_nodes:
        x,y=nodes_index2global_nodes_coord[key]
        draw.ellipse((y - r, x - r, y + r, x + r), fill=(255,0,0), width=1)

    if plot:
    
        plt.imshow(image)
        plt.show()
        
        fig,axes = plt.subplots(1,1,figsize=(14,7))
        
        
        
        axes.imshow(img_drawings)
        
        axes.text(img_aligned.shape[1]/2-25,border_eps/3,'entry nodes', color=(0, 0, 1),fontsize=15)
        axes.text(img_aligned.shape[1]/2-25,img_aligned.shape[0] - border_eps/3,'exit nodes', color=(1, 0, 0),fontsize=15)
        axes.arrow(12, border-5, 0, img_aligned.shape[0] - border_eps, length_includes_head=True, head_width=10, head_length=10,color=(0,0,0))
        axes.text(2, img_aligned.shape[0]*2/3,'hit direction',rotation=90, color=(0, 0, 0),fontsize=15)
        
        # plt.xlim(0,200)
        # plt.ylim(0,50)
        
        # plt.savefig('miniature_figures_aligned.png', bbox_inches='tight')
        
        plt.show()
        
    return entry_nodes, exit_nodes, num_of_nodes, img_aligned, nodes_index2global_nodes_coord, cnts



# Graph buiding 

In [ ]:
def create_crack_graph(img_shape, num_of_nodes, nodes_index2global_nodes_coord, cnts,  eps=100, line_eps=3,border = 30, border_eps = 0,border_number_min = 2,border_pixel=255,same_node_eps = 5, plot=False):
        
    g = nx.DiGraph()
    image_node_coord2node_index = np.zeros(img_shape,dtype=np.int32)
    for key in range(num_of_nodes):
        x,y=nodes_index2global_nodes_coord[key]
        image_node_coord2node_index[x,y]=key
        g.add_node(key, pos=(y,x)) 
    
    m=[]
    a=[]
    
    
    img_contours = grainDraw.draw_contours(Image.fromarray(np.zeros((img_shape[0],img_shape[1]))), color_line = (255), cnts=cnts,corners = False)
    img_contours_np = np.array(img_contours)
    
    for start_node_index in tqdm(range(num_of_nodes)):
        
        # choose cell
        start_node_x,start_node_y=nodes_index2global_nodes_coord[start_node_index]
        
        # for rectangular vertical slice only!
        ###############################################

        # left y slice border
        if start_node_y-eps<0:
            left_border_y=0
        else:
            left_border_y=start_node_y-eps
    
        # right y slice border
        if start_node_y+eps>image_node_coord2node_index.shape[1]:
            right_border_y=image_node_coord2node_index.shape[1]-1
        else:
            right_border_y=start_node_y+eps
    
        # upper_border
        if start_node_x+eps>image_node_coord2node_index.shape[0]-1:
            upper_border=image_node_coord2node_index.shape[0]-1
        else:
            upper_border=start_node_x+eps
            
        map_slice = image_node_coord2node_index[start_node_x+1:upper_border,left_border_y:right_border_y]
        ###############################################
        
        nodes_indices_indices = np.where(map_slice.flatten()!=0)
        nodes_indices = map_slice.flatten()[nodes_indices_indices]
        
        # next node search
        for node_index in nodes_indices:
        
            end_node_x, end_node_y = nodes_index2global_nodes_coord[node_index]
    
            if abs(end_node_x-start_node_x)>same_node_eps or abs(end_node_y-start_node_y)>same_node_eps:
            
                ab = LineString([(start_node_x, start_node_y), (end_node_x, end_node_y)])
                left = ab.parallel_offset(line_eps, 'left')
                left_p,_ = np.array(left.coords)
                perp_v = np.array((start_node_x-left_p[0],start_node_y-left_p[1]))
                perp_v = perp_v/np.linalg.norm(perp_v)
                
                mean_border_pixels=0
    
                    
                for p in range(0 - line_eps, 1 + line_eps):
                    line_coords=np.array(list(bresenham(np.round(start_node_x+p*perp_v[0]).astype(np.int32),
                                                        np.round(start_node_y+p*perp_v[1]).astype(np.int32),
                                                        np.round(end_node_x+p*perp_v[0]).astype(np.int32),
                                                        np.round(end_node_y+p*perp_v[1]).astype(np.int32)
                                                       )))
                    
                    line_coords_pixels=img_contours_np[line_coords[:,0],line_coords[:,1]][2:-2]
                    border_pixels_num = np.where(line_coords_pixels==border_pixel)[0].shape[0]
                    if border_pixels_num<=border_eps:
                        mean_border_pixels+=1
    
                if mean_border_pixels>=border_number_min and start_node_index!=node_index:
    
                    g.add_edge(start_node_index,node_index, weight=np.linalg.norm((end_node_x-start_node_x, end_node_y-start_node_y)))

    if plot:
        img_tmp = img_contours.convert('RGB')
        img_tmp = 255 - np.array(img_tmp)
        dark_pixels = img_tmp[:,:,0]==0
        img_tmp[dark_pixels]=[255,0,0]
        
        pos = nx.get_node_attributes(g, 'pos')
        fig,axes = plt.subplots(1,1,figsize=(15,15))
        
        nx.draw(g, pos, ax =axes,  with_labels=True, node_color='lightblue', node_size=500, font_size=15)
        axes.imshow(img_tmp, cmap='gray')
        # axes.imshow(np.full_like(img_tmp,255), cmap='gray')
        
        # axes[0].invert_yaxis()
        # axes[1].invert_yaxis()
        
        plt.axis("on")
        axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        
        axes.arrow(12, border-5, 0, 200, width=0.3, length_includes_head=True, head_width=10, head_length=10,color=(0,0,0))
        
        axes.text(2, img_aligned.shape[0]*2/3,'hit direction',rotation=90, color=(0, 0, 0),fontsize=15)
        axes.text(img_aligned.shape[1]/2-25, border/3,'entry nodes', color=(0, 0, 1),fontsize=15)
        axes.text(img_aligned.shape[1]/2-25,img_aligned.shape[0] - border/3,'exit nodes', color=(1, 0, 0),fontsize=15)
        
        # plt.savefig('graph.jpg',bbox_inches='tight')
        # plt.savefig('graph_no_borders.jpg',bbox_inches='tight')
        plt.show()

    return g

# Algorithm debugging

In [ ]:
eps=100
line_eps = 2
border_eps = 0
border_number_min = 1
border_pixel=255

test_show_eps=10

start_node_index = 20
node_index = 21

start_node_x,start_node_y=nodes_index2global_nodes_coord[start_node_index]
end_node_x, end_node_y = nodes_index2global_nodes_coord[node_index]
print(start_node_x,start_node_y)
print(end_node_x, end_node_y)


ab = LineString([(start_node_x, start_node_y), (end_node_x, end_node_y)])
left = ab.parallel_offset(line_eps, 'left')
left_p,_ = np.array(left.coords)
perp_v = np.array((start_node_x-left_p[0],start_node_y-left_p[1]))
perp_v = perp_v/np.linalg.norm(perp_v)

mean_border_pixels=0

test_map=np.zeros_like(img_contours_np)

n=[]
for p in range(0 - line_eps, 1 + line_eps):
    line_coords=np.array(list(bresenham(np.round(start_node_x+p*perp_v[0]).astype(np.int32),
                                        np.round(start_node_y+p*perp_v[1]).astype(np.int32),
                                        np.round(end_node_x+p*perp_v[0]).astype(np.int32),
                                        np.round(end_node_y+p*perp_v[1]).astype(np.int32)
                                       )))
    
    line_coords_pixels=img_contours_np[line_coords[:,0],line_coords[:,1]][2:-2]
    border_pixels_num = np.where(line_coords_pixels==border_pixel)[0].shape[0]

    n.append(line_coords_pixels)
        
    print(border_pixels_num)
    if border_pixels_num<=border_eps:
        mean_border_pixels+=1

    test_map[line_coords[:,0],line_coords[:,1]]=170

test_map[start_node_x,start_node_y]=255
test_map[end_node_x,end_node_y]=255

plt.figure(figsize=(10,10))
plt.imshow(test_map[start_node_x-test_show_eps:end_node_x+test_show_eps,start_node_y-test_show_eps:end_node_y+test_show_eps])

print('result', mean_border_pixels)
if mean_border_pixels>=border_number_min and start_node_index!=node_index:
    print('added')


# Graph traversal

In [ ]:
%%time

mem1 = psutil.virtual_memory().available
def find_paths(entry_node, exit_node):
    all_paths=[]
    all_entry_nodes=[]
    all_exit_nodes=[]
    all_path_len_edges=[]
    all_path_len_pixels=[]
    
    paths = list(nx.all_simple_paths(g, source=entry_node, target=exit_node))

    for path in paths: 
        all_paths.append(path)
        all_entry_nodes.append(int(entry_node))
        all_exit_nodes.append(int(exit_node))
        all_path_len_edges.append(len(path))
        l=nx.path_weight(g,path, 'weight')
        all_path_len_pixels.append(l)

    return pd.DataFrame({'path':all_paths,
                         'path_len_edges':all_path_len_edges,
                         'path_len_pixel':all_path_len_pixels,
                         'entry_node':all_entry_nodes,
                         'exit_node':all_exit_nodes
                        })

workers = 23
cart_list=[entry_nodes, exit_nodes]
cart_list=[element for element in itertools.product(*cart_list)]

with WorkerPool(n_jobs=workers) as pool:
    results = pool.map(find_paths, cart_list, progress_bar= True)

df = pd.concat(results,axis=0)

mem2 = psutil.virtual_memory().available
print('entry_nodes', entry_nodes)
print('exit_nodes', exit_nodes)
print('used memory Gib',(mem1-mem2)/1024**3)

In [ ]:
%%time

mem1 = psutil.virtual_memory().available
def find_paths(entry_node, exit_node):
    all_paths=[]
    all_entry_nodes=[]
    all_exit_nodes=[]
    all_path_len_edges=[]
    all_path_len_pixels=[]
    
    os.system(f'/bin/bash -c "bfs_custom/bfs_paths_dev bfs_custom/200x500.txt {entry_node} {exit_node} ./tmp/" ')
    paths = 0

    # for path in paths: 
    #     all_paths.append(path)
    #     all_entry_nodes.append(int(entry_node))
    #     all_exit_nodes.append(int(exit_node))
    #     all_path_len_edges.append(len(path))
    #     l=nx.path_weight(g,path, 'weight')
    #     all_path_len_pixels.append(l)

    # return pd.DataFrame({'path':all_paths,
    #                      'path_len_edges':all_path_len_edges,
    #                      'path_len_pixel':all_path_len_pixels,
    #                      'entry_node':all_entry_nodes,
    #                      'exit_node':all_exit_nodes
    #                     })

    return 

workers = 200
cart_list=[entry_nodes, exit_nodes]
cart_list=[element for element in itertools.product(*cart_list)]

with WorkerPool(n_jobs=workers) as pool:
    results = pool.map(find_paths, cart_list, progress_bar= True)

# df = pd.concat(results,axis=0)

mem2 = psutil.virtual_memory().available
print('entry_nodes', entry_nodes)
print('exit_nodes', exit_nodes)
print('used memory Gib',(mem1-mem2)/1024**3)

In [ ]:
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 255)  # Default to 255 if not provided
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value

def pad_image(image, pad_size, pad_value=255):
    return np.pad(image, pad_size, mode=pad_with, padder=pad_value)

pad_size = ((2, 2), (3, 3)) 

padded_image = pad_image(image, pad_size, pad_value=0)

print(image.shape)
print(padded_image.shape)

plt.imshow(padded_image)

In [ ]:
img_aligned.shape

In [ ]:
# image = io.imread('../datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-001.jpeg')[:520,:500]
# image = io.imread('WC+8Co_5_fixed_001.jpg')[:300,:800]
image = io.imread('../datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-003.jpeg')[:300,:500]

#  некорректно
# image = io.imread('../datasets/original/o_bc_left/Ultra_Co25/Ultra_Co25-001.jpeg')[:200,:200]


entry_nodes, exit_nodes, num_of_nodes, img_aligned, nodes_index2global_nodes_coord, cnts = preprocess_image(image,plot=True)

g = create_crack_graph(img_aligned.shape, num_of_nodes,nodes_index2global_nodes_coord,cnts,  plot=False)

# nx.write_edgelist(g,'350x500.txt',data=False)

# save graph object to file
# pickle.dump(g, open('g.pickle', 'wb'))


#200

# entry_nodes [18, 30, 11, 17, 0, 10]
# exit_nodes [83, 45, 46, 47, 57, 58, 59]

# 250

# entry_nodes [18, 30, 11, 17, 0, 10]
# exit_nodes [81, 82, 83, 78, 79, 74, 75]

# 300

# entry_nodes [17, 28, 11, 16, 0, 10]
# exit_nodes [96, 97, 84, 85, 76, 77]

# 350 

# entry_nodes [17, 28, 11, 16, 0, 10]
# exit_nodes [111, 112, 101, 102, 84, 85]

# plt.imshow(img_aligned)

In [ ]:
# print('entry_nodes', entry_nodes)
# print('exit_nodes', exit_nodes)

In [ ]:
!cat /proc/meminfo | grep Mem

In [ ]:
!nvcc -O2 bfs_custom/bfs_paths_dev.cu -o bfs_custom/bfs_paths_dev 
!nvcc -O2 bfs_custom/bfs_paths.cu -o bfs_custom/bfs_paths

In [ ]:
!bfs_custom/bfs_paths bfs_custom/300x500.txt 0 97 ./tmp/

In [ ]:
with open('bfs_paths_15_110_300x500.json') as f:
    data = json.load(f)

In [ ]:
data.__len__()

In [ ]:
!cat /proc/meminfo | grep Mem

In [ ]:
# df[(df.entry_node==0) & (df.exit_node==75) ]

image.shape

In [ ]:
# image = io.imread('../datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-001.jpeg')[:520,:500]
# image = io.imread('WC+8Co_5_fixed_001.jpg')[:300,:800]
# image = io.imread('../datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-003.jpeg')[:200,:500]

i=1
step=300

image = io.imread('WC+8Co_5_crack_clear.jpg')[100+int(i*step):100+int(i+1)*step,:1000]

#  некорректно
# image = io.imread('../datasets/original/o_bc_left/Ultra_Co25/Ultra_Co25-001.jpeg')[:200,:200]


entry_nodes, exit_nodes, num_of_nodes, img_aligned, nodes_index2global_nodes_coord, cnts, orig_img_padded = preprocess_image(image,plot=True)

g = create_crack_graph(img_aligned.shape, num_of_nodes,nodes_index2global_nodes_coord,cnts,  plot=False)

# nx.write_edgelist(g,'350x500.txt',data=False)

# save graph object to file
# pickle.dump(g, open('g.pickle', 'wb'))

In [ ]:
plt.figure(figsize=(15,7))
plt.imshow(orig_img_padded,cmap='gray')

In [ ]:
cart_list=[entry_nodes, exit_nodes]
cart_list=[element for element in itertools.product(*cart_list)]

# np.save('pairs.npy',np.array(cart_list))

# Graph info

In [ ]:
mean_path_length_edges = df['path_len_edges'].mean()
mean_degree  = np.array(g.degree)[:,1].mean()
entry_nodes_num = len(entry_nodes)
exit_nodes_num = len(exit_nodes)
number_of_nodes = len(g.nodes)

cart_list=[entry_nodes, exit_nodes]
cart_list=[element for element in itertools.product(*cart_list)]

print(f'number of paths {len(df)/1e6:.2}*10^6')
print('-----')
print(f'number of entry nodes n={entry_nodes_num}' )
print(f'number of exit nodes k={exit_nodes_num}')
print(f'mean node degree p={mean_degree:.2f}')
print(f'mean path length edges L={mean_path_length_edges:.2f}')
print(f'number of nodes t={number_of_nodes}')
print(f'number of edges t={len(g.edges)}')
print(f'number  of pairs={len(cart_list)}')

m = (number_of_nodes-entry_nodes_num-exit_nodes_num)/mean_path_length_edges
print(f'm={m:.2f}')

N = entry_nodes_num*exit_nodes_num*(mean_degree/2 * mean_degree/2)**m
print(f'N={N/1e6:.2f}*10^6')

# Shortest paths plot

In [ ]:
df=pd.read_csv('paths.csv')

entry_nodes = np.unique(df['entry_node'])
exit_nodes = np.unique(df['exit_node'])

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    row = df[df.entry_node==entry_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_entry_paths.append(row)

shortest_exit_paths = []
for exit_node in tqdm(exit_nodes):
    row = df[df.exit_node==exit_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_exit_paths.append(row)

df_shortest_entry = pd.DataFrame(shortest_entry_paths)
df_shortest_exit = pd.DataFrame(shortest_exit_paths)

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(15,15))

pos = nx.get_node_attributes(g, 'pos')

for i in range(len(df_shortest_entry)):
    line = df_shortest_entry.iloc[i]
    path = line['path']
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color='blue',width=1)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['entry_node']],node_color='blue',node_size = 100)

for i in range(len(df_shortest_exit)):
    line = df_shortest_exit.iloc[i]
    path = list(reversed(line['path']))
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color='r',width=1)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['exit_node']],node_color='r',node_size = 100)

axes.imshow(img_aligned, cmap='gray')

plt.axis("on")
axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

axes.arrow(12, border-5, 0, 200, width=0.3, length_includes_head=True, head_width=10, head_length=10,color=(0,0,0))

axes.text(2, img_aligned.shape[0]*2/3,'hit direction',rotation=90, color=(0, 0, 0),fontsize=15)
axes.text(img_aligned.shape[1]/2-25, border/3,'entry nodes', color=(0, 0, 1),fontsize=15)
axes.text(img_aligned.shape[1]/2-25,img_aligned.shape[0] - border/3,'exit nodes', color=(1, 0, 0),fontsize=15)

# plt.savefig('shortests_paths.jpg',bbox_inches='tight')

plt.show()

In [ ]:
df_shortest_entry['path'] = df_shortest_entry['path'].apply(lambda x: np.array(x))
df_shortest_exit['path'] = df_shortest_exit['path'].apply(lambda x: np.array(x))

df_shortest_entry['path_len_pixel'] = df_shortest_entry['path_len_pixel'].apply(lambda x: int(x))
df_shortest_exit['path_len_pixel'] = df_shortest_exit['path_len_pixel'].apply(lambda x: int(x))

df_shortest_entry['entry_node'] = df_shortest_entry['entry_node'].apply(lambda x: int(x))
df_shortest_exit['entry_node'] = df_shortest_exit['entry_node'].apply(lambda x: int(x))

df_shortest_entry['exit_node'] = df_shortest_entry['exit_node'].apply(lambda x: int(x))
df_shortest_exit['exit_node'] = df_shortest_exit['exit_node'].apply(lambda x: int(x))

df_shortest_entry['path_len_edges'] = df_shortest_entry['path_len_edges'].apply(lambda x: int(x))
df_shortest_exit['path_len_edges'] = df_shortest_exit['path_len_edges'].apply(lambda x: int(x))

df_shortest_entry.to_csv('shortest_entry.csv')
df_shortest_exit.to_csv('shortest_exit.csv')

# Plot distances hists

In [ ]:
def plot_paths_dist(data, name, title=False, folder='tmp', save=True, bins = 60, xlim = None):
    std=np.std(data)
    mean=np.mean(data)
    
    plt.figure(figsize=(7,5))
    plt.hist(data, bins=bins)
    
    plt.axvline(mean-2*std,color=(0.2,0.5,0.5))
    plt.axvline(mean-std,color=(1,1,0))
    plt.axvline(mean,color=(1,0,0))
    plt.axvline(mean+std,color=(0,1,1))
    plt.axvline(mean+2*std,color=(0,1,0.5))
    if title:
        plt.title(title)
    plt.legend(['mean-2*std','mean-std','mean','mean+std','mean+2*std',f'paths len\nnumber of paths\n{len(data)/10**6:.3f}*10^6\nmean={mean:.2f}\nstd={std:.2f}'])
    plt.xlabel('length of crack, pixels')
    
    if xlim:
        plt.xlim(xlim[0],xlim[0])
    
    plt.savefig(f'{folder}/{name}', bbox_inches='tight')
    plt.show()


pixel = 50 / 1000

save=False
# save=True

folder = "tmp_plots"
Path(folder).mkdir(parents=True, exist_ok=True)

names = []

# all paths pixels 
data=df['path_len_pixel']
name = f"entry_exit_paths_all_pixels.jpg"
names.append(name)
plot_paths_dist(data, name, folder = folder, save=save)

# exit paths pixels 
for i,exit_node in tqdm(enumerate(exit_nodes)):
    name = f"entry_exit_paths_{i}_pixels.jpg"
    names.append(name)
    data=df[df['exit_node']==exit_nodes[i]]['path_len_pixel']
    plot_paths_dist(data, name,title=f'exit node {exit_node}', folder=folder, save=save)

# all paths edges 
data=df['path_len_edges']
name =  f"entry_exit_paths_all_edges.jpg"
names.append(name)
plot_paths_dist(data, name, folder=folder, save=save, bins=30)

# exit paths edges 
for i,exit_node in tqdm(enumerate(exit_nodes)):
    name = f"entry_exit_paths_{i}_edges.jpg"
    names.append(name)
    data=df[df['exit_node']==exit_nodes[i]]['path_len_edges']
    plot_paths_dist(data, name,title=f'exit node {exit_node}', folder = folder, save=save, bins=30)

shutil.make_archive('plots', 'zip', folder)

# Dynamic grid search (not done)

img_viztmp_img1) image_nodes_coord2nodes_index
2) nodes_index2global_nodes_coord
3) image_node_coord2node_index

1) grid_cell_coord2grid_cell_index
2) grid_cell_index2grid_cell_coord
3) image_coord2grid_cell_index

In [ ]:
# process nodes of graph
# add all nodes to graph
g = nx.DiGraph()
image_node_coord2node_index = np.zeros(img_shape)
for key in range(num_of_nodes):
    x,y=nodes_index2global_nodes_coord[key]
    image_node_coord2node_index[x,y]=key
    g.add_node(key, pos=(x,y))
    
# pos = nx.get_node_attributes(g, 'pos')
# plt.figure(figsize = (5,5))
# nx.draw(g, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)

In [ ]:
# create grid
cell_size=10
grid_size=np.int32(img_shape/cell_size)

number_of_cells=0
grid_cell_coord2grid_cell_index={}
grid_cell_index2grid_cell_coord={}
image_coord2grid_cell_index = np.zeros(img_shape)
for xi in range(grid_size[0]):
    for yi in range(grid_size[1]):   
        grid_cell_coord2grid_cell_index[(xi,yi)]=number_of_cells
        grid_cell_index2grid_cell_coord[number_of_cells]=(xi,yi)
        
        # map of grid cell indices
        image_coord2grid_cell_index[xi*cell_size:(xi+1)*cell_size,
                                 yi*cell_size:(yi+1)*cell_size
                                ]=np.full((cell_size,cell_size), number_of_cells)
        number_of_cells+=1

# plt.imshow(image_coord2grid_cell_index[:10,:100])    

grid = np.array(sliding_window_view(image_node_coord2node_index, (cell_size, cell_size))[::cell_size, ::cell_size])
grid_summed = grid.sum(axis=(2,3))
plt.imshow(grid_summed,cmap='gray', origin='lower')

In [ ]:
def check_borders(point, shape):
    if point[0]>=0 and point[1]>=0 and point[0]<=shape[0] and point[1]<=shape[1]:
        return True
    else:
        return False
K=10
img_viz_numpy=np.array(img_aligned)

#127, 255

m=[]
# grid search
for start_node_index in tqdm(range(num_of_nodes)):
    
    # choose cell
    start_node_x,start_node_y=nodes_index2global_nodes_coord[start_node_index]
    grid_cell_id=image_coord2grid_cell_index[start_node_x,start_node_y]
    cell_x,cell_y=grid_cell_index2grid_cell_coord[grid_cell_id]
    
    if cell_x-K//2<0:
        left_border_x=K//2-cell_x
    else:
        left_border_x=cell_x-K//2-2

    # right x slice border
    if cell_y+K//2>grid.shape[0]:
        right_border_x=grid.shape[0]
    else:
        right_border_x=cell_y+K//2+2

    # upper_border
    if cell_y+K>grid.shape[1]-1:
        upper_border=grid.shape[1]-1
    else:
        upper_border=cell_y+K

    grid_summed_slice=grid_summed[left_border_x:right_border_x,cell_y:upper_border]
    summed_grid_cell_x,summed_grid_cell_y=np.where(grid_summed_slice!=0)
    grid_slice=grid[summed_grid_cell_x,summed_grid_cell_y]

    nodes_indices_indices = np.where(grid_slice.flatten()!=0)
    nodes_indices =grid_slice.flatten()[nodes_indices_indices]

    for node_index in (nodes_indices):
        end_node_x,end_node_y=nodes_index2global_nodes_coord[node_index]
        mean_pixels=[]
        for p in range(-2,3):
            line_coords=np.array(list(bresenham(start_node_x+p, start_node_y,end_node_x+p, end_node_y)))
            mean_pixel=np.mean(img_viz_numpy[line_coords[:,0],line_coords[:,0]])
            m.append(mean_pixel)
            mean_pixels.append(mean_pixel)
        line_mean=np.mean(mean_pixels)
        if line_mean<160 or line_mean>222:
            g.add_edge(start_node_index,node_index)
        # m.append(np.mean(mean_pixels))
   

In [ ]:
plt.imshow(img_viz_numpy,cmap='gray', origin='lower')

In [ ]:
pos = nx.get_node_attributes(g, 'pos')
plt.figure(figsize = (5,5))
nx.draw(g, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)

In [ ]:
plt.hist(m,bins=20)

In [ ]:
np.array(list(bresenham(-1, -4, 3, 2)))

In [ ]:
#     for xi in range(-K//2,K//2+1):
#         cell_x_tmp = cell_x + xi
#         for yi in range(0,K):
#             cell_y_tmp = cell_y + yi
            
#             if check_borders((cell_x_tmp,cell_y_tmp),img_shape):



    # wave_flag=True
    # wave=0
    # indices=[]

    # while wave_flag:
        # center cell
#         center_point=[node_x, node_y+1]
        
#         if check_borders(center_point):
#             indices.append(center_point)
        
#         # border cells
#         for i in range(wave+1):
#             left_point=[node_x-1-i, node_y-1-i]
#             right_point=[node_x+1+i, node_y+1+i]
            
#         if check_borders(left_point):
#             indices.append(left_point)
            
#         if check_borders(right_point):
#             indices.append(right_point)

# Deprecated

Entry points 22
number of nodes 1770

In [ ]:
# contour_points = cnts[3]
gs=[]


for i,contour_points in enumerate(cnts):
    g = nx.Graph()

    for i, point in enumerate(contour_points):
        g.add_node(i, pos=point)


    for i in range(len(contour_points) - 1):
        g.add_edge(i, i + 1)

    g.add_edge(len(contour_points) - 1, 0)
    gs.append(g)

names = tuple([f"g{i}-" for i in range(len(cnts))])

g=nx.union_all(gs,rename=names)

pos = nx.get_node_attributes(g, 'pos')

plt.figure(figsize = (10,10))
nx.draw(g, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
plt.show()

In [ ]:
G = nx.dorogovtsev_goltsev_mendes_graph(3)
nx.draw(G, with_labels=True, node_color='lightblue', node_size=500, font_size=10)

In [ ]:
# G = nx.path_graph(5)

# input 22 points
# output 66 points
# G = nx.complete_multipartite_graph(4,5)
G = nx.dorogovtsev_goltsev_mendes_graph(7)
# nx.draw(G, with_labels=True, node_color='lightblue', node_size=500, font_size=10)

G.number_of_nodes()

In [ ]:
# white_img = np.zeros_like((orig_img_padded))
# white_img = np.ascontiguousarray(white_img, dtype=np.uint8)
# cnts_adj = [cnt.reshape((-1,1,2)) for cnt in cnts ]
# img_contours_np = cv2.drawContours(white_img,cnts_adj,-1,(255),2).astype(np.uint8)

# plt.figure(figsize=(15,5))
# plt.imshow(img_contours_np[:200,:500],cmap='gray')
# plt.show()


# ------------# 

# img_contours = draw_edges(np.zeros_like(orig_img_padded), cnts=cnts, r=2, l_width=1)
# img_contours_np = np.array(img_contours)

# plt.figure(figsize=(15,5))
# plt.imshow(img_contours_np[:200,:500],cmap='gray')
# plt.show()